<a href="https://colab.research.google.com/github/a-woodbury/RxVision/blob/master/Notebooks/RxVision_Technical_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RxVision

RxVision is a image recognition model for identifying medications. The goal is to create a fast, reliable, and scalable solution for clinicians, dispensing pharmacies, and patients to get real-time identification of a capsule or tablet. 

This first version uses a convolutional neural network to distinguish images of medications into 15 classes, in this case specific medications by their National Drug Code (NDC). 

The 15 drugs selected for training were chosen for their ability to be evaluated in the real-world; patients (my friends and family) provided (willingly) images of their medications in their hand or on their counter to showcase how a patient would use this model in reality. Each drug class provided 30 NIH images for training and validation and 1 real-world image for testing. 

The training images were acquired from the NIH dataset, which houses over 130,000 images of 4300+ distinct medications. On average, each medication has 29 images that can be used for training, but there are about 250 with over 50. images. I would have preferred to train on those medications, but no real-world images were available.


# Model Prep


## Import Packages

In [5]:
import pandas as pd
import requests
from IPython.display import Image
from ftplib import FTP #needed to make the request to the server

# packages for processing images
! pip install rawpy
from PIL import Image
from PIL import ImageFile
import rawpy
import imageio
ImageFile.LOAD_TRUNCATED_IMAGES = True


import os, shutil, sys #required for moving files

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from pathlib import Path


import imageio
import imgaug as ia

import pandas as pd
import numpy as np
import pickle
import seaborn as sns

import os
import sys
import warnings
warnings.filterwarnings('ignore')
#!{sys.executable} -m pip install opencv-python

import time
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

#import tensorflow as tf
#from tensorflow import keras

import scipy
from PIL import Image
from scipy import ndimage

#import tensorflow as tf
#from tensorflow import keras
import tensorflow.keras 
from keras import layers
from keras import models
from keras import optimizers
from keras import models
from keras import layers
import multiprocessing

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from keras.models import Model
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.applications.resnet import ResNet50
from keras.callbacks import CSVLogger

from sklearn.model_selection import RandomizedSearchCV, cross_val_score, GridSearchCV, validation_curve 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, r2_score, recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.pipeline import make_pipeline
#from tensorflow.keras import get_default_graph

from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# This code has been tested with TensorFlow 1.6
#import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
np.random.seed(123)

# Transfer learning with VGG16
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.vgg16 import preprocess_input

import random 

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn import metrics
import seaborn as sns

from numpy import loadtxt
from keras.models import load_model


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Functions

In [6]:
def model_scores(model):
  '''Return validation accuracy and real-world accuracy after evaluating provided model'''
  model_val_results = model.evaluate(val_generator)
  modelacc = model_val_results[1]
  model_rw_results = model.evaluate(realworld_generator)
  modelrw = model_rw_results[1]
  print('\nValidation Accuracy: ' + str(int(modelacc*100)) + '%' + '\nReal-world Accuracy: ' + str(int(modelrw*100)) + '%')

def model_acc_val_plot(model):
  '''Return train and validation loss and accuracy plots from hist log'''
  class_size =  int(val_generator.samples / len(val_generator.class_indices))
  acc = hist['acc']
  val_acc = hist['val_acc']

  loss = hist['loss']
  val_loss = hist['val_loss']

  epochs_range = range(EPOCHS)

  plt.figure(figsize=(16, 8))
  plt.subplot(1, 2, 1)
  plt.plot(range(0,len(hist)), acc, label='Training Accuracy')
  plt.plot(range(0,len(hist)), val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 3, 3)
  plt.plot(range(0,len(hist)), loss, label='Training Loss')
  plt.plot(range(0,len(hist)), val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.savefig('../Images/{}_AccLoss.png'.format(model.name))
  plt.show()

def model_confusion(model):
  '''Return a confusion matrix for predictions made from evaluating the proviced model'''
  labels = list((val_generator.class_indices).values())
  pred = model.predict(val_generator)
  y_pred=np.argmax(pred,axis=1)
  y_true = val_generator.classes

  cf_matrix = metrics.confusion_matrix(y_true, y_pred, labels=labels)

  fig, ax = plt.subplots(figsize=(8,6.75))  
  pal = sns.light_palette("#ffab40", as_cmap=True)
  sns.heatmap(cf_matrix, annot=True,cmap=pal,ax=ax)
  plt.ylabel('Class Actual', fontweight='bold')
  plt.xlabel('Class Predicted', fontweight='bold')
  plt.title('../Images/{}_Confusion Matrix'.format(model.name), fontweight='bold', loc='left')
  plt.savefig('../Images/{}_conf'.format(model.name))

def predict_plot(model):
  '''
  Return an image plot with class, number of correct predictions,
  and predicted classes (ordered by descending frequency)
  '''
  labels = list((val_generator.class_indices).values())
  pred = model.predict(val_generator)
  y_pred=np.argmax(pred,axis=1)
  y_true = val_generator.classes
  class_size =  int(val_generator.samples / len(val_generator.class_indices))
  dfx = df[df.TYPE == 'MC_COOKED_CALIBRATED_V1.2']
  samplesdfx = dfx.groupby(['NDC']).min().reset_index()
  sampleslist2 = samplesdfx.FILE.tolist()
  #len(sampleslist2)

  samplefiles = []
  for image in sampleslist2:
      smplsplt = image.split('/')
      keep = smplsplt[-1]
      keep = keep[:-4]
      keep= keep +('.JPG')
      samplefiles.append(keep)

      
  #for file in os.listdir():
  drgimg = os.listdir()
  images = []
  for file in samplefiles:
      data = plt.imread(file)
      images.append(data)
  plt.figure(figsize=(15,15))
  columns = 5

  for i, image in enumerate(images): # iterate through the images in the array 'images'
      k = i * class_size
      j = (i + 1) * class_size
      trues = int(y_true[k:j].mean())
      preds = list(y_pred[k:j]).count(trues)
      lst = list(y_pred[k:j])
      preddict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0}
      for pred in lst:
          preddict[pred] +=1
      preddict = {k: v for k, v in sorted(preddict.items(), key=lambda item: item[1], reverse=True)}
      for k, v in list(preddict.items()):
          if v == 0:
              del preddict[k]
      predlist = list(preddict.keys())
      dname = df.DRUG[df.FILENAME.str.contains(samplefiles[i][:-4])].tolist()[0] # get the drug name from the df for the image in index i 
      dndc = df.NDC[df.FILENAME.str.contains(samplefiles[i][:-4])].tolist()[0] # get the NDC from the df for the image in index i 
      title = '[' + str(i) + '] ' + ' ' + dname + '\nCorrect: ' + str(preds) + '\nPreds: ' +  ', '.join(map(str,predlist)) # title for each subplot: class, drug name, and NDC
      plt.subplot(len(images) / columns + 1, columns, i + 1)
      plt.suptitle('RxID15 Classes',fontweight='bold',fontsize='large', color= '#ffab40')
      plt.subplots_adjust(hspace=0.2,wspace=0.25, top=.9, bottom=.2) # i believe this is the subplots spacing from each other and within the plot
      plt.margins(tight=True) # not sure which margins this is impacting
      plt.title(title,fontweight='semibold',fontsize='10')
      plt.imshow(image)
      plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
      plt.savefig('/content/drive/My Drive/RxID2/Images/{}_predictions.jpg'.format(model.name),format='jpg',quality=95,dpi=300, bbox='tight',pad_inches = 0) # bbox has always given me the output i wanted...
  %cd ../..

def real_world_predicts(model):
  '''Return predicted classes for real world images using provided model'''
  pred = model.predict(realworld_generator)
  y_pred=np.argmax(pred,axis=1)
  print('\nReal-world predictions 0-14: ', y_pred)

## Data Selection




The dataset includes a [readme](ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Pills/AAREADME) that outlines the dataset



In [9]:
typdict = {'NDC':'str'}
df = pd.read_csv('/content/drive/My Drive/RxID2/directory_of_images.txt',sep='|', dtype=typdict, names=['NDC','PART_#','FILE','TYPE','DRUG'])
df

,NDC,PART_#,FILE,TYPE,DRUG
0,00002322730,1,PillProjectDisc69/images/CLLLLUPGIX7J8MP1WWQ9W...,C3PI_Reference,STRATTERA 10MG
1,00002322730,1,PillProjectDisc98/images/PRNJ-AXZIQ!HUQKJJBP_D...,C3PI_Reference,STRATTERA 10MG
2,00002322730,1,PillProjectDisc10/images/79U-YY6M1UUR6F127ZMAC...,C3PI_Test,STRATTERA 10MG
3,00002322730,1,PillProjectDisc11/images/7WVFV5H74!ELFNQ_GUH92...,C3PI_Test,STRATTERA 10MG
4,00002322730,1,PillProjectDisc20/images/B4CH0R9B7PEQ6GORRX-8X...,C3PI_Test,STRATTERA 10MG
...,...,...,...,...,...
133769,99207046730,1,PillProjectDisc103/images/TY5OVXLLOXV6H4I1TDVT...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133770,99207046730,1,PillProjectDisc31/images/BEIR3XK38EMGSDOZTWMUK...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133771,99207046730,1,PillProjectDisc69/images/CLJ1W40OS0XG5H6IVYT!N...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133772,99207046730,1,PillProjectDisc77/images/CSUHWDZ!XAZSEJHDANMFR...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB


In [ ]:
ndcs = [    
'00009033102',    
'65862019430',
'00591084510',
'55111068305',
'00527134410',
'00093005801',    
'45802091987',
'61958070101',
'49884003501',
'00591078005',
'31722020701',
'68180047901',
'65862007701',
'00054472825',
'68180040301'
]
ndcs_pack = [x[:-2] for x in ndcs]

In [ ]:
df = df.dropna()
df.DRUG = df.DRUG.str.upper()
df[['ORIG_FOLDER','IMAGES','FILENAME']] = df.FILE.str.split('/', expand=True)
df['FILETYPE'] = df.FILENAME.str[-4:]
df = df[df.FILETYPE != '.WMV']# will remove video files from query
df['NDC_prod'] = df.NDC.str[:-2]
df = df[df.NDC_prod.isin(ndcs_pack)]

In [ ]:
df.NDC[df.NDC == '00093005805'] = '00093005801'

df.DRUG[df.NDC == '00093005801'] = 'LEVOTHYROXINE 0.088MG'
df

In [ ]:
df.DRUG.value_counts()

In [ ]:
df

,NDC,IDK,FILE,TYPE,DRUG,ORIG_FOLDER,IMAGES,FILENAME,FILETYPE,NDC_prod
2946,00009033102,1,PillProjectDisc58/images/CB4CEJKI72-2IAAF8SPNK...,C3PI_Reference,CLEOCIN 75MG,PillProjectDisc58,images,CB4CEJKI72-2IAAF8SPNK-YD6QHSBHE.JPG,.CR2,000090331
2947,00009033102,1,PillProjectDisc95/images/MJ8SIXGLA!IDK6QKOJQ8N...,C3PI_Reference,CLEOCIN 75MG,PillProjectDisc95,images,MJ8SIXGLA!IDK6QKOJQ8N5DOBZIKHE.JPG,.CR2,000090331
2949,00009033102,1,PillProjectDisc107/images/XW27OGQ!GSTCS6SVCFSE...,C3PI_Test,CLEOCIN 75MG,PillProjectDisc107,images,XW27OGQ!GSTCS6SVCFSE6F!WHA7PYT.JPG,.JPG,000090331
2950,00009033102,1,PillProjectDisc13/images/9CLLUNVVKAJ4Y!Q0R_4_H...,C3PI_Test,CLEOCIN 75MG,PillProjectDisc13,images,9CLLUNVVKAJ4Y!Q0R_4_H0I4EVZG5C.JPG,.JPG,000090331
2951,00009033102,1,PillProjectDisc17/images/B18QCJP3ZMIWFPGXJZ4R3...,C3PI_Test,CLEOCIN 75MG,PillProjectDisc17,images,B18QCJP3ZMIWFPGXJZ4R3I-OL2GR6Z-.JPG,.JPG,000090331
...,...,...,...,...,...,...,...,...,...,...
127776,68180047901,1,PillProjectDisc42/images/BP4V9ZT0LMIB4BO872E-W...,MC_COOKED_CALIBRATED_V1.2,SIMVASTATIN,PillProjectDisc42,images,BP4V9ZT0LMIB4BO872E-WK025340X84.JPG,.PNG,681800479
127777,68180047901,1,PillProjectDisc51/images/BXRU3YZEHJU82Z4XML43I...,MC_COOKED_CALIBRATED_V1.2,SIMVASTATIN,PillProjectDisc51,images,BXRU3YZEHJU82Z4XML43IK7X1JWYWH7.JPG,.PNG,681800479
127778,68180047901,1,PillProjectDisc72/images/CN_3M1A3P5IO95ONJAKUI...,MC_COOKED_CALIBRATED_V1.2,SIMVASTATIN,PillProjectDisc72,images,CN_3M1A3P5IO95ONJAKUIFJYQC18Y9_.JPG,.PNG,681800479
127779,68180047901,1,PillProjectDisc83/images/CY8E7A05V71HWIUH2IQOZ...,MC_COOKED_CALIBRATED_V1.2,SIMVASTATIN,PillProjectDisc83,images,CY8E7A05V71HWIUH2IQOZYMM-2BJSD8.JPG,.PNG,681800479


In [ ]:
df.to_csv('../Data/rxid15.csv')

# Modeling

In [2]:
df = pd.read_csv('../Data/rxid15.csv')

In [3]:
train_folder = './RxID2_split/train'
val_folder = './RxID2_split/validation'
realworld_folder = './RxID2_split/test'

In [7]:
BATCH_SIZE = 16
IMG_SHAPE  = 28 
EPOCHS = 50


val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_folder,
                                                                       shuffle=False,
                                                                       #class_mode='binary',
                                                                       target_size=(IMG_SHAPE,IMG_SHAPE),
                                                                       batch_size = BATCH_SIZE,
                                                                       classes = [cls for cls in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, cls))][:15])

train_generator = ImageDataGenerator(rescale=1./255,
                                    ).flow_from_directory(train_folder,
                                                                          shuffle=True,
                                                                   #class_mode='binary',
                                                                          target_size=(IMG_SHAPE,IMG_SHAPE),
                                                                          batch_size=BATCH_SIZE,
                                                                          classes=[cls for cls in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, cls))][:15])

realworld_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(realworld_folder,
                                                                       shuffle=False,
                                                                       class_mode='categorical',
                                                                       target_size=(IMG_SHAPE,IMG_SHAPE),
                                                                       batch_size = BATCH_SIZE,
                                                                       classes=[cls for cls in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, cls))][:15])
# labels = list((val_generator.class_indices).values())

Found 70 images belonging to 15 classes.


Found 297 images belonging to 15 classes.
Found 69 images belonging to 15 classes.


## Matching Networks

### Setup

In [8]:
import numpy as np
from tensorflow.keras.layers import Input, Lambda, Conv2D, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

# Assuming each batch has a set of support samples and a single query sample
# N is the number of support samples per class
N = 5

# Support and query inputs
support_input = Input(shape=(15*N, IMG_SHAPE, IMG_SHAPE, 3), name="support_input")
query_input = Input(shape=(IMG_SHAPE, IMG_SHAPE, 3), name="query_input")

# Embedding function (using the CNN structure you provided)
def embed_model(input_shape):
    model = models.Sequential(name='embedding_model')
    model.add(layers.Conv2D(128, (3,3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((3, 3)))
    model.add(layers.Conv2D(64, (3,3), activation='relu', padding="same"))
    model.add(layers.Flatten())
    return model

# Getting embeddings for support and query sets
embed = embed_model((IMG_SHAPE, IMG_SHAPE, 3))
support_embeddings = embed(support_input)
query_embedding = embed(query_input)

# Cosine similarity between the support and query embeddings
def cosine_similarity(a, b):
    normalize_a = tf.nn.l2_normalize(a, axis=-1)
    normalize_b = tf.nn.l2_normalize(b, axis=-1)
    cos_similarity = tf.reduce_sum(tf.multiply(normalize_a, normalize_b), axis=-1)
    return cos_similarity

# Computing the similarities and predictions
similarities = Lambda(lambda x: cosine_similarity(x[0], x[1]))([support_embeddings, query_embedding])
weights = tf.nn.softmax(similarities)
predictions = Lambda(lambda x: tf.matmul(tf.expand_dims(x[0], axis=1), x[1]))([weights, support_embeddings])

# Building the final model
matching_model = Model(inputs=[support_input, query_input], outputs=predictions)
matching_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
matching_model.summary()

NameError: name 'x_train' is not defined

### Training

In [ ]:
history = matching_network.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=val_generator,
                    callbacks=[early,csv_logger],
                    # use_multiprocessing=True,
                    # workers=multiprocessing.cpu_count(),
                    verbose=2)
matching_network.save('../Data/Models/model_{}'.format(matching_network.name))

### Analysis

In [ ]:
hist = pd.read_csv('../Data/Models/hist_{}.log'.format(matching_network.name), sep=',', engine='python')

model_scores(matching_network)

real_world_predicts(matching_network)

In [ ]:
model_acc_val_plot(matching_network)

In [ ]:
model_confusion(matching_network)

In [ ]:
predict_plot(matching_network)

## Siamese Networks

### Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

# Define the embedding model
def embedding_model(input_shape):
    model = tf.keras.models.Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((3, 3)),
        Conv2D(64, (3, 3), activation='relu', padding="same"),
        Flatten()
    ])
    return model

# Define the Euclidean distance function
def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# Assume `input_shape` is the shape of your images
input_shape = (IMG_SHAPE, IMG_SHAPE, 3)
base_network = embedding_model(input_shape)

# Create the left input and point to the base network
input_a = Input(shape=input_shape)
vect_output_a = base_network(input_a)

# Create the right input and point to the base network
input_b = Input(shape=input_shape)
vect_output_b = base_network(input_b)

# Measure the similarity of the two vector outputs
output = Lambda(euclidean_distance)([vect_output_a, vect_output_b])

# Specify the model
siamese_model = Model([input_a, input_b], output)

# Compile the model
siamese_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Set the callbacks
csv_logger = CSVLogger('../Data/Models/hist_{}.log'.format(model.name), separator=',', append=False)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

# Train the model
siamese_model.fit([x_train_1, x_train_2], y_train, epochs=10, callbacks=[csv_logger, early])

# Evaluate the siamese_model
test_loss, test_acc = siamese_model.evaluate([x_test_1, x_test_2], y_test)

### Training

In [ ]:
history = siamese_model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=val_generator,
                    callbacks=[early,csv_logger],
                    # use_multiprocessing=True,
                    # workers=multiprocessing.cpu_count(),
                    verbose=2)
siamese_model.save('../Data/Models/model_{}'.format(siamese_model.name))

### Analysis

In [ ]:
hist = pd.read_csv('../Data/Models/hist_{}.log'.format(siamese_model.name), sep=',', engine='python')

model_scores(siamese_model)

real_world_predicts(siamese_model)

In [ ]:
model_acc_val_plot(siamese_model)

In [ ]:
model_confusion(siamese_model)

In [ ]:
predict_plot(siamese_model)

## Relational Networks

### Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

# Define the object processing model
def object_processing_model(input_shape):
    model = tf.keras.models.Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((3, 3)),
        Conv2D(64, (3, 3), activation='relu', padding="same"),
        Flatten(),
        Dense(64, activation='relu')
    ])
    return model

# Define the relation processing model
def relation_processing_model():
    model = tf.keras.models.Sequential([
        Dense(64, activation='relu'),
        Dense(15, activation='softmax')
    ])
    return model

# Assume `input_shape` is the shape of your images
input_shape = (IMG_SHAPE, IMG_SHAPE, 3)
object_model = object_processing_model(input_shape)

# Create the two inputs and process them through the object model
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = object_model(input_a)
processed_b = object_model(input_b)

# Concatenate the processed objects and feed them to the relation model
concatenated = Concatenate(axis=-1)([processed_a, processed_b])
relation_model = relation_processing_model()
output = relation_model(concatenated)

# Specify the model
relational_model = Model([input_a, input_b], output)

# Compile the model
relational_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# Set the callbacks
csv_logger = CSVLogger('../Data/Models/hist_{}.log'.format(relational_model.name), separator=',', append=False)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

# Train the model
relational_model.fit([x_train_1, x_train_2], y_train, epochs=10, callbacks=[csv_logger, early])

# Evaluate the model
test_loss, test_acc = relational_model.evaluate([x_test_1, x_test_2], y_test)


### Training

In [ ]:
history = relational_model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=val_generator,
                    callbacks=[early,csv_logger],
                    # use_multiprocessing=True,
                    # workers=multiprocessing.cpu_count(),
                    verbose=2)
relational_model.save('../Data/Models/model_{}'.format(relational_model.name))

### Analysis

In [ ]:
hist = pd.read_csv('../Data/Models/hist_{}.log'.format(relational_model.name), sep=',', engine='python')

model_scores(relational_model)

real_world_predicts(relational_model)

In [ ]:
model_acc_val_plot(relational_model)

In [ ]:
model_confusion(relational_model)

In [ ]:
predict_plot(relational_model)